<h1>Imports</h1>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset

import numpy as np

from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

import torchvision
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision import datasets
from torchvision.transforms import ToTensor

from tqdm import tqdm

import tensorflow as tf

import random

<h1>Basic Visualization Checks</h1>
<p>Pull up a random image to make sure everything is appearing correctly.</p>

In [ ]:
# Define Dataset Path
dataDir = './data'
dataType = 'train2017'
annFile = f'{dataDir}/annotations/instances_{dataType}.json'

# Initialize COCO
coco = COCO(annFile)

# Get Category ID for Person
catIds = coco.getCatIds(catNms=['person'])

# Get Image IDs for Person Images
imgIds = coco.getImgIds(catIds=catIds)

# Load Random Image
img = coco.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]

# Display Random Image
image_path = f"{dataDir}/{dataType}/{img['file_name']}"
image = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Load Image Annotations
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)

# Draw Boxes
for ann in anns:
    bbox = ann['bbox']
    x, y, width, height = bbox
    cv2.rectangle(image, (int(x), int(y)), (int(x+width), int(y+height)), (255, 0, 0), 2)

# Display Image with Boxes
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

<h1>Preprocess the Data</h1>


In [ ]:
class PersonCocoDetection(CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super().__init__(root, annFile, transform)
        self.coco = COCO(annFile)
        self.ids = self._filter_person_ids()

    def _filter_person_ids(self):
        person_ids = []
        person_category_id = self.coco.getCatIds(catNms=['person'])[0]
        for img_id in self.ids:
            ann_ids = self.coco.getAnnIds(imgIds=img_id, catIds=person_category_id, iscrowd=None)
            if len(ann_ids) > 0:
                person_ids.append(img_id)
        return person_ids
    
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        target = [ann for ann in target if ann['category_id'] == 1]
        return img, target


def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = []
    for item in batch:
        target = item[1]
        for ann in target:
            ann['bbox'] = torchvision.ops.box_convert(torch.tensor(ann['bbox']), 'xywh', 'xyxy').tolist()
        new_target = {
            'boxes': torch.tensor([ann['bbox'] for ann in target], dtype=torch.float32),
            'labels': torch.tensor([ann['category_id'] for ann in target], dtype=torch.int64),
            'image_id': torch.tensor([ann['image_id'] for ann in target], dtype=torch.int64),
            'area': torch.tensor([ann['area'] for ann in target], dtype=torch.float32),
            'iscrowd': torch.tensor([ann['iscrowd'] for ann in target], dtype=torch.int64)
        }
        targets.append(new_target)
    return images, targets


# Define Transformation
transform = T.Compose([
    T.ToTensor()
])


dataDir = './data'
dataType = 'train2017'
annFile = f'{dataDir}/annotations/instances_{dataType}.json'
dataset = PersonCocoDetection(root=f'{dataDir}/{dataType}', annFile=annFile, transform = transform)

# Create Subset
subset_size = 20000  
random_seed = 42  

# Set Seed
random.seed(random_seed)
subset_indices = random.sample(range(len(dataset)), subset_size)
subset = Subset(dataset, subset_indices)

train_dataloader = DataLoader(subset, batch_size=16, shuffle=True, collate_fn=collate_fn)

<h1>Model Selection</h1>

In [ ]:
# Load Model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=True)

# Input Features
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace Head with Person Detection Model
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

# Training Parameters
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1) 

<h1>Train Model</h1>

In [ ]:
# Training Loop
num_epochs = 20
patience = 5
best_loss = float('inf')
epochs_no_improve = 0

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}/{num_epochs}") as pbar:
        for i, data in enumerate(train_dataloader):
            images, targets = data
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            try:
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                epoch_loss += losses.item()

                losses.backward()
                optimizer.step()
            except Exception as e:
                print(f"An error occurred: {e}")
                continue

            pbar.update(1)
            pbar.set_postfix(loss=losses.item())

        lr_scheduler.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_epoch_loss}")

    # Early Stop
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break

<h1>Visualize Validation Image</h1>

In [ ]:
# Define Database Path
dataDir = './data'
dataType = 'val2017'
annFile = f'{dataDir}/annotations/instances_{dataType}.json'

# Initialize COCO
coco = COCO(annFile)

# Get Cat IDs for Person
catIds = coco.getCatIds(catNms=['person'])

# Get Person Images
imgIds = coco.getImgIds(catIds=catIds)

# Load Random Image
img = coco.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]

# Display Random Image
image_path = f"{dataDir}/{dataType}/{img['file_name']}"
image = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Load Image Annotations
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)

# Draw Boxes
for ann in anns:
    bbox = ann['bbox']
    x, y, width, height = bbox
    cv2.rectangle(image, (int(x), int(y)), (int(x+width), int(y+height)), (255, 0, 0), 2)

# Display Image with Boxes
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

<h1>Validation</h1>

In [ ]:
# Load Model State
model.load_state_dict(torch.load('20K_fasterrcnn_resnet50_fpn.pth'))

# Set Model Device
model.to(device)

# Eval Mode
model.eval()

# Load Validation Dataset
dataDir = './data'
dataType = 'val2017'
annFile = f'{dataDir}/annotations/instances_{dataType}.json'
val_dataset = PersonCocoDetection(root=f'{dataDir}/{dataType}', annFile=annFile, transform=transform)

# Create Subset
subset_size = 10  
random_seed = 42  

# # Set Seed
random.seed(random_seed)
subset_indices = random.sample(range(len(val_dataset)), subset_size)
val_subset = Subset(val_dataset, subset_indices)

# Create Dataloader
test_dataloader = DataLoader(val_subset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Evaluate Model
with torch.no_grad():
    print("Starting evaluation...")
    for batch_idx, (images, targets) in enumerate(tqdm(test_dataloader, desc="Evaluating")):
        print(f"Processing batch {batch_idx+1}/{len(test_dataloader)}")
        
        images = [img.to(device) for img in images]
        print(f"Moved images to {device}")
        
        # Get Predictions
        outputs = model(images)
        print("Obtained outputs from the model")

        for i, image in enumerate(images):
            plt.figure(figsize=(12, 8))
            plt.imshow(image.permute(1, 2, 0).cpu().numpy())
            ax = plt.gca()
            
            boxes = outputs[i]['boxes'].cpu().numpy()
            scores = outputs[i]['scores'].cpu().numpy()
            print(f"Image {i+1} - boxes: {len(boxes)}, scores: {len(scores)}")

            # Draw Boxes
            for box, score in zip(boxes, scores):
                if score > 0.5: 
                    x_min, y_min, x_max, y_max = box
                    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(rect)
                    plt.text(x_min, y_min, f'{score:.2f}', color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))
            
            plt.axis('off')
            plt.show()
            print(f"Finished displaying image {i+1} of batch {batch_idx+1}")

    print("Evaluation complete")